# Segmenting and Clustering Neighborhoods in Toronto

### Create a new dataframe to store data from wiki page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
columns = ['PostalCode','Borough','Neighborhood']
toronto_postal = pd.DataFrame(columns = columns)

### Fetch the data from source and collect necessary data

In [35]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
Canada_data = BeautifulSoup(response.text, 'lxml')
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto_postal = toronto_postal.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [36]:
toronto_postal = toronto_postal[toronto_postal.Borough!='Not assigned']
toronto_postal = toronto_postal[toronto_postal.Borough!= 0]
toronto_postal.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto_postal.shape[0]):
    if toronto_postal.iloc[i][2] == 'Not assigned':
        toronto_postal.iloc[i][2] =toronto_postal.iloc[i][1]
        i = i+1
                                 
df = toronto_postal.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


### Cleaning data

Drop "None" rows, replace "Not Assigned with NaN"

In [22]:
df = df.dropna()

In [38]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


In [39]:
df.shape

(103, 3)